<a href="https://colab.research.google.com/github/rhiats/classification_CAD/blob/main/Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Heart Disease**

In [2]:
pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
heart_disease = fetch_ucirepo(id=45)

# data (as pandas dataframes)
X = heart_disease.data.features
y = heart_disease.data.targets

In [4]:
y.head()

,num
0,0
1,2
2,1
3,0
4,0


In [5]:
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0


In [6]:
print(heart_disease.metadata)

{'uci_id': 45, 'name': 'Heart Disease', 'repository_url': 'https://archive.ics.uci.edu/dataset/45/heart+disease', 'data_url': 'https://archive.ics.uci.edu/static/public/45/data.csv', 'abstract': '4 databases: Cleveland, Hungary, Switzerland, and the VA Long Beach', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 303, 'num_features': 13, 'feature_types': ['Categorical', 'Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['num'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1989, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C52P4X', 'creators': ['Andras Janosi', 'William Steinbrunn', 'Matthias Pfisterer', 'Robert Detrano'], 'intro_paper': {'ID': 231, 'type': 'NATIVE', 'title': 'International application of a new probability algorithm for the diagnosis of coronary artery disease.', 'authors': 'R. Detrano, A. Jánosi, W. Steinbrunn, M

In [7]:
heart_disease.variables

,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,None,years,no
1,sex,Feature,Categorical,Sex,None,None,no
2,cp,Feature,Categorical,None,None,None,no
3,trestbps,Feature,Integer,None,resting blood pressure (on admission to the ho...,mm Hg,no
4,chol,Feature,Integer,None,serum cholestoral,mg/dl,no
5,fbs,Feature,Categorical,None,fasting blood sugar > 120 mg/dl,None,no
6,restecg,Feature,Categorical,None,None,None,no
7,thalach,Feature,Integer,None,maximum heart rate achieved,None,no
8,exang,Feature,Categorical,None,exercise induced angina,None,no
9,oldpeak,Feature,Integer,None,ST depression induced by exercise relative to ...,None,no


In [8]:
list(heart_disease.variables['description'])

[None,
 None,
 None,
 'resting blood pressure (on admission to the hospital)',
 'serum cholestoral',
 'fasting blood sugar > 120 mg/dl',
 None,
 'maximum heart rate achieved',
 'exercise induced angina',
 'ST depression induced by exercise relative to rest',
 None,
 'number of major vessels (0-3) colored by flourosopy',
 None,
 'diagnosis of heart disease']

In [9]:
X = X[['age','sex','trestbps','chol','fbs','thalach','exang','oldpeak','ca']]

In [10]:
X.head()

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,ca
0,63,1,145,233,1,150,0,2.3,0.0
1,67,1,160,286,0,108,1,1.5,3.0
2,67,1,120,229,0,129,1,2.6,2.0
3,37,1,130,250,0,187,0,3.5,0.0
4,41,0,130,204,0,172,0,1.4,0.0


**Data Set Overview**

In [35]:
print(f"Numer of patients: {X.shape[0]}")

Numer of patients: 303


**Class Imbalance**

**Age Distribution and T-Test**

**Sex Bar Graph**

In [44]:
#1 is male
gender_cnt_df=X[['sex','age']].groupby('sex').count().reset_index()
gender_cnt_df.rename(columns={'age': 'Count'}, inplace=True)
gender_cnt_df

,sex,Count
0,0,97
1,1,206


**Resting Blood Pressure Distribution and T-Test**

**Cholesterol Distribution and T-Test**

**Fasting Blood Sugar Bar Graph**

**Exercise Induced Angina Bar Graph**

**ST Depression Distribution and T-Test**

**Bar Graph Number of Major Vessels Fluorescing**

**Linearly Separable Equation**

In [26]:
import numpy as np

df = X[['age','trestbps','chol','exang','oldpeak']]
df["y"] = y

# Assume X is (n_samples, n_features), y is (n_samples,)
class_0 = df[df["y"] == 0]
class_1 = df[df["y"] > 0]

centroid_0 = np.mean(class_0, axis=0)
centroid_1 = np.mean(class_1, axis=0)

print("Centroid Class 0:", centroid_0)
print("Centroid Class 1:", centroid_1)

Centroid Class 0: age          52.585366
trestbps    129.250000
chol        242.640244
exang         0.140244
oldpeak       0.586585
y             0.000000
dtype: float64
Centroid Class 1: age          56.625899
trestbps    134.568345
chol        251.474820
exang         0.546763
oldpeak       1.574101
y             2.043165
dtype: float64


<ipython-input-26-302dc631ea50>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["y"] = y


In [27]:
w = centroid_1 - centroid_0  # direction vector

In [28]:
midpoint = (centroid_0 + centroid_1) / 2  # midpoint is the decision boundary

In [29]:
print("Midpoint:", midpoint)
print("Direction vector (normal):", w)

Midpoint: age          54.605633
trestbps    131.909173
chol        247.057532
exang         0.343503
oldpeak       1.080343
y             1.021583
dtype: float64
Direction vector (normal): age         4.040533
trestbps    5.318345
chol        8.834576
exang       0.406519
oldpeak     0.987515
y           2.043165
dtype: float64


In [31]:
def predict_linear_separator(X, w, midpoint):
    projections = (X - midpoint) @ w
    return (projections > 0).astype(int)  # class 1 if on centroid_1's side

y_pred = predict_linear_separator(df.drop(columns=['y'], axis=1), w, midpoint)

accuracy = np.mean(y_pred == df["y"])
print(f"Centroid-separator accuracy: {accuracy:.2f}")

Centroid-separator accuracy: 0.54


**Maximum Heart Rate Achieved Distribution**

****

**Logistic Regression**